In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [12]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("iabhishekofficial/mobile-price-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/mobile-price-classification


In [13]:
import pandas as pd

train = pd.read_csv(os.path.join(path, "train.csv"))
test = pd.read_csv(os.path.join(path, "test.csv"))

print(train.head())
print(test.head())

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   
1        136        3  ...        905      1988  2631    17     3          7   
2        145        5  ...       1263      1716  2603    11     2          9   
3        131        6  ...       1216      1786  2769    16     8         11   
4        141        2  ...       1208      1212  1411     8     2         15   

   three_g  touch_screen  wifi  price_

#### CSV Dataset

In [14]:
# Load mobile price dataset into dataframe
mobile_df = pd.read_csv(os.path.join(path, "train.csv")).dropna()
print(mobile_df.price_range.unique())
mobile_df.head()

[1 2 3 0]


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [15]:
# Seperate features and target
mb_X = mobile_df.drop(columns=['price_range']).values.astype(np.float32)
mb_y = mobile_df.price_range.values

# Standardize features
scaler = StandardScaler()
mb_X = scaler.fit_transform(mb_X)
# ^^^^^^^^

# Convert to tensors
mb_X = torch.tensor(mb_X)
mb_y = torch.tensor(mb_y)

# Create train and test datasets
mb_train_set = TensorDataset(mb_X, mb_y)
train_split = 0.8
mb_train_size = int(train_split * len(mb_train_set))
mb_test_size = len(mb_train_set) - mb_train_size
mb_train_set, mb_test_set = torch.utils.data.random_split(mb_train_set, [mb_train_size, mb_test_size])

# Create dataloaders
mb_train_loader = DataLoader(mb_train_set, batch_size=16, shuffle=True)
mb_test_loader = DataLoader(mb_test_set, batch_size=16, shuffle=True)

print(len(mb_train_set), len(mb_test_set))
print(len(mb_train_loader), len(mb_test_loader))

1600 400
100 25


#### Functions and Models for use

In [16]:
class MLP_CSV(nn.Module):
    def __init__(self):
        super(MLP_CSV, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=20, out_features=128), # 20 input features
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
        )
        self.fc3 = nn.Sequential(
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
        )
        self.out = nn.Linear(in_features=32, out_features=4)  # 4 output for classification

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.out(x)
        return x

class MLP_MNIST(nn.Module):
    def __init__(self):
        super(MLP_MNIST, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=8*8, out_features=32), # 8*8 = 64 resized input pixels
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=32, out_features=24),
            nn.ReLU(),
        )
        self.fc3 = nn.Sequential(
            nn.Linear(in_features=24, out_features=16),
            nn.ReLU(),
        )
        self.out = nn.Linear(in_features=16, out_features=10)  # 10 output for numbers

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.out(x)
        return x

In [17]:
def training_loop(train_dl, test_dl, model, n_epochs=200):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Your optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.CrossEntropyLoss()
    # ^^^^^^^^

    n_show_loss = n_epochs // 10 if n_epochs > 10 else 1
    train_history = {"train_loss":[], "validate_loss":[],
                     "train_acc":[], "validate_acc":[]}
    # Begin training loop
    for epoch in range(n_epochs):
        # Train loop
        train_loss, train_correct = 0, 0
        model.train(True)
        for X_t, y_t in train_dl:
            X_t, y_t = X_t.to(device), y_t.to(device)
            optimizer.zero_grad()
            y_pred = model(X_t)
            # This is your training loss calculation
            loss = loss_fn(y_pred, y_t)
            loss.backward()
            # ^^^^^^^^
            optimizer.step()
            train_loss += loss.item()
            train_correct += (y_pred.argmax(1) == y_t).float().sum().item()
        train_history["train_loss"].append(train_loss / len(train_dl))
        train_history["train_acc"].append(train_correct / len(train_dl.dataset))

        # Test loop
        valid_loss, valid_correct = 0, 0
        model.eval()
        with torch.no_grad():
            for X_v, y_v in test_dl:
                X_v, y_v = X_v.to(device), y_v.to(device)
                y_predv = model(X_v)
                vloss = loss_fn(y_predv.squeeze(), y_v.squeeze())
                valid_loss += vloss.item()
                valid_correct += (y_predv.argmax(1) == y_v).float().sum().item()
        train_history["validate_loss"].append(valid_loss / len(test_dl))
        train_history["validate_acc"].append(valid_correct / len(test_dl.dataset))

        if ((epoch+1) % n_show_loss == 0) or (epoch == n_epochs-1):
            print(f"Epochs {epoch+1}".ljust(12),
                  f"train loss {train_history['train_loss'][-1]:.6f}",
                  f"valid loss {train_history['validate_loss'][-1]:.6f}",
                  f"train acc {train_history['train_acc'][-1]:.6f}",
                  f"valid acc {train_history['validate_acc'][-1]:.6f}")

    return train_history

def plot_result(history):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figwidth(10)
    fig.suptitle("Train vs Validation")
    ax1.plot(history["train_acc"], label="Train")
    ax1.plot(history["validate_acc"], label="Validation")
    ax1.legend()
    ax1.set_title("Accuracy")

    ax2.plot(history["train_loss"], label="Train")
    ax2.plot(history["validate_loss"], label="Validation")
    ax2.legend()
    ax2.set_title("Loss")
    fig.show()


----
## HW
----
### - CSV Dataset
Mobile price dataset ที่กำหนดมีข้อมูลเบื้องต้น ดังนี้
- จำนวน Features = 20 (RAM, Internal memory, Battery, ...)
- Target คือ `price_range` โดยมีทั้งหมด 4 ค่า คือ 0, 1, 2, 3 (Low cost, Medium cost, High cost, Very high cost)

> กำหนดให้ใช้โมเดล `MLP_CSV` เป็นต้นแบบ การ train ให้ใช้จำนวน `epochs` = 200, `optimizer` = Adam, `learning_rate` = 0.0001, `train_split` = 0.8, `batch_size` = 16, ไม่ใช้ Learning rate scheduler

'

1.) ให้ทดลอง train โมเดลโดยใช้ training loop ที่ให้มาหรือเขียนขึ้นเอง แบบธรรมดาไม่มีการปรับเพิ่มเติม ใช้เพื่อเป็นผลลัพธ์ Baseline สำหรับข้อต่อ ๆ ไป

- > แคปรูปผลลัพธ์ใหม่ 1 รูป

2.) ให้ทดลอง train โมเดลโดยใช้ L2 Regularizer เพิ่มในข้อ 1.) รวมทั้งหมด 2 แบบ คือ
  1. ใช้ L2 แบบ Marginal loss โดย `lambda` = [0.03, 0.01, 0.005] (เพิ่ม L2 loss ใน Training loop)
  2. ใช้ L2 แบบ Weight decay โดย `weight_decay` = [0.06, 0.01, 0.005] (ปรับ param ใน `optimizer`)

- > แคปรูปผลลัพธ์ใหม่ทั้งหมด 6 รูป พร้อมกับแสดงส่วนที่แก้ไขเพื่อทำ Marginal loss ภายใน Training loop และ วิเคราะห์เปรียบเทียบผลลัพธ์ที่ได้ทั้งหมดกับข้อ 1.)

3.) ให้ทดลอง train โมเดลโดยใช้ Dropout เพิ่มในข้อ 1.) ภายในชั้น fc1, fc2 และ fc3 และแทนค่า `p` ในแต่ละชั้นเป็นค่าดังต่อไปนี้
  1. fc1, fc2, fc3 = 0.5, 0.3, 0.2
  2. fc1, fc2, fc3 = 0.3, 0.2, 0.1
  3. fc1, fc2, fc3 = 0.2, 0.2, 0.2

- > แคปรูปผลลัพธ์ใหม่ทั้งหมด 3 รูป และ วิเคราะห์เปรียบเทียบผลลัพธ์ที่ได้กับข้อ 1.)

4.) ให้ทดลอง train โมเดล `MLP_CSV` ต้นแบบจากข้อ 1.) อีกครั้ง พร้อมกับ train โมเดลที่ใช้ BatchNorm เพิ่มในข้อ 1.) ภายในชั้น fc1, fc2 และ fc3 (ใช้ `nn.BatchNorm1d`) แต่จะมีการปรับในส่วนการเตรียม Mobile price Dataset ดังนี้
  1. โมเดลที่ใช้ BatchNorm ใช้ Dataset ที่ **ผ่าน** การทำ Standardized ด้วย `StandardScaler()` เทียบกับผลลัพธ์ข้อ 1.)
  2. โมเดล `MLP_CSV` ต้นแบบจากข้อ 1.) ใช้ Dataset ที่ **ไม่ผ่าน** การทำ Standardized
  3. โมเดลที่ใช้ BatchNorm ใช้ Dataset ที่ **ไม่ผ่าน** การทำ Standardized เทียบกับผลลัพธ์ข้อ 4.2)

- > แคปรูปผลลัพธ์ใหม่ทั้งหมด 3 รูป และ วิเคราะห์เปรียบเทียบผลลัพธ์ที่ได้

'

----

### - Torchvision Dataset

MNIST dataset จาก torchvision
- ภาพตัวเลขเขียนด้วยลายมือ
- ขนาด 28x28 pixel จะได้จำนวน features = 784
- Target = เลข 0 - 9 (10 ตัว)

```python
import torchvision
from torchvision import transforms
from torchvision.transforms import v2
import ...

# Your own transform
transforms = v2.Compose([
                v2.ToImage(),
                v2.ToDtype(torch.float32, scale=True),
                ...,
                ...])

# Optimizer, criterion (loss), and other hyper-pars
# model = ...
# optimizer = ...
# criterion = ...
# batch_size = ...
# scheduler = ...
# other = ...

# Download train-valid(test) Dataset
mnist_train_data = torchvision.datasets.MNIST('./Image/train_directory/', transform=transforms, download=True, train=True)
mnist_train_loader = torch.utils.data.DataLoader(mnist_train_data, batch_size=batch_size, shuffle=True)

mnist_test_data = torchvision.datasets.MNIST('./Image/test_directory/', transform=transforms, download=True, train=False)
mnist_valid_loader = torch.utils.data.DataLoader(mnist_test_data, batch_size=batch_size, shuffle=True)

# training loop
for epoch in range(epochs):
    model.train()
    # ...
    
    for inputs, labels in mnist_train_loader:
    # or
    for i, (inputs, labels) in enumerate(mnist_train_loader):
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      # ...update...

    # Optional
    model.eval()
    with torch.no_grad():
      for vinputs, vlabels in mnist_valid_loader:
      # or
      for j, (vinputs, vlabels) in enumerate(mnist_valid_loader):
      voutputs = model(vinputs)
      loss = criterion(voutputs, vlabels)
      # ...non-update...

```

'

5.) ให้ทำการสร้าง Dataset ที่ใช้การ `transforms` เพิ่มเติมจากที่กำหนดไว้ โดยให้ลดขนาดภาพลงเหลือ 8x8 pixels (64 features) และเพิ่มการ Transform อีกอย่างน้อย 1 แบบ พร้อมกับสร้าง Dataloaders ที่มีจำนวน train : test = 80 : 20 ของจำนวน Dataset ทั้งหมดและใช้ `batch_size` = 16
* > แคปรูปส่วนกำหนดการ transforms และการทำ Dataloaders

ุ6.) ให้ทดลอง train โมเดล `MLP_MNIST` ต้นแบบที่กำหนด และ train โมเดลที่นักศึกษาออกแบบเอง (หรือเพิ่มต่อจากต้นแบบก็ได้) โดยกำหนดให้เพิ่มส่วนเทคนิคการใช้ L2 Regularizer, Dropout และ BatchNorm ภายในโมเดลที่ออกแบบเอง (เลือกอย่างน้อย 2 เทคนิค จะใส่ในทุก Hidden layers หรือไม่ก็ได้)

กำหนดไว้ว่าโมเดลที่นักศึกษาออกแบบ จะสามารถปรับจำนวน features ภายในได้อิสระ แต่ต้องมีจำนวน Hidden layers เท่ากับ `MLP_MNIST` ต้นแบบ และกำหนดการ train จะต้องใช้จำนวน `epochs` เท่ากัน โดยเลือกเองให้ <= 50, `optimizer` เลือกเอง และสามารถใช้ Learning rate scheduler ได้

* > จงทดลองหาผลลัพธ์การทำงาน ที่ทำให้ได้ประสิทธิภาพสูงกว่า `MLP_MNIST` ต้นแบบ เช่น Accuracy สูงกว่าเดิม
>
> แคปรูปผลลัพธ์การทำงาน โครงสร้างโมเดลสุดท้าย และอธิบายว่าใช้เทคนิคอะไร ทำอย่างไรบ้างที่ทำให้ได้ประสิทธิภาพสูงขึ้น